In [15]:
import numpy as np
from numpy.polynomial import Polynomial


In [32]:
def ls_american_option_quadratic_iter(X, t, r, strike):
    # given no prior exercise we just receive the payoff of a European option
    cashflow = np.maximum(strike - X[-1, :], 0.0)
    # iterating backwards in time
    for i in reversed(range(1, X.shape[0] - 1)):
        # discount factor between t[i] and t[i+1]
        df = np.exp(-r * (t[i + 1] - t[i]))
        # discount cashflows from next period
        cashflow = cashflow * df
        x = X[i, :]
        # exercise value for time t[i]
        exercise = np.maximum(strike - x, 0.0)
        # boolean index of all in-the-money paths
        itm = exercise > 0
        # fit polynomial of degree 2
        print(x[itm], cashflow[itm])
        fitted = Polynomial.fit(x[itm], cashflow[itm], 2)
        print(fitted)
        # approximate continuation value
        continuation = fitted(x)
        print("c", continuation)
        # boolean index where exercise is beneficial
        ex_idx = itm & (exercise > continuation)
        # update cashflows with early exercises
        cashflow[ex_idx] = exercise[ex_idx]

        yield cashflow, x, fitted, continuation, exercise, ex_idx


def longstaff_schwartz_american_option_quadratic(X, t, r, strike):
    for cashflow, *_ in ls_american_option_quadratic_iter(X, t, r, strike):
        print(cashflow)
        pass
    return cashflow.mean(axis=0) * np.exp(-r * (t[1] - t[0]))

In [33]:
t = np.linspace(0, 3, 4)
r = 0.06
strike = 1.1
X = np.array([
    [1.00, 1.09, 1.08, 1.34],
    [1.00, 1.16, 1.26, 1.54],
    [1.00, 1.22, 1.07, 1.03],
    [1.00, 0.93, 0.97, 0.92],
    [1.00, 1.11, 1.56, 1.52],
    [1.00, 0.76, 0.77, 0.90],
    [1.00, 0.92, 0.84, 1.01],
    [1.00, 0.88, 1.22, 1.34]
]).T

In [34]:
longstaff_schwartz_american_option_quadratic(X, t, r, strike)

[1.08 1.07 0.97 0.77 0.84] [0.         0.06592352 0.16951762 0.18835291 0.08475881]
0.13792605 - 0.05761432·x - 0.04357117·x²
c [ 0.03674056 -0.19012381  0.04589834  0.11752682 -0.82938608  0.15196921
  0.15641792 -0.12955348]
[0.         0.         0.06592352 0.13       0.         0.33
 0.26       0.        ]
[1.09 0.93 0.76 0.92 0.88] [0.         0.12242939 0.3107823  0.24485878 0.        ]
0.11284536 - 0.13628979·x + 0.03692953·x²
c [ 0.01348511 -0.00635402 -0.01277862  0.10874928  0.00646033  0.28606468
  0.11700927  0.15276213]
[0.         0.         0.06208443 0.17       0.         0.34
 0.18       0.22      ]


0.11443433004505696

In [50]:
x = [0, 1, -1, 2, -2, 3, -3,]
y = [0, 1, 1, 4, 4, 9 ,9]
p = Polynomial.fit(x, y, 2)
p

Polynomial([2.11817704e-15, 0.00000000e+00, 9.00000000e+00], domain=[-3.,  3.], window=[-1.,  1.], symbol='x')

In [51]:
print(p)

2.11817704e-15 + 0.0·x + 9.0·x²
